In [2]:
# First cell - Import required libraries
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import faiss
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional

# Second cell - Create the ConversationManager class
class ConversationManager:
    def __init__(self, model_name: str):
        # Initialize the sentence transformer for creating embeddings
        self.tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.embedding_size = 384
        self.index = faiss.IndexFlatL2(self.embedding_size)
        self.conversations = []
        self.ollama_model = model_name
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message to the conversation history and create its embedding"""
        inputs = self.tokenizer(message, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.pooler_output.cpu().detach().numpy()
        self.index.add(embeddings)
        self.conversations.append({'role': role, 'content': message})
    
    def generate_response(self, message: str, temperature: float = 0.7) -> str:
        """Generate a response using Ollama and maintain conversation history"""
        self.add_message(message, 'user')
        
        # Use the last 10 messages for context
        messages = self.conversations[-10:]
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history"""
        self.conversations = []
        self.index = faiss.IndexFlatL2(self.embedding_size)

# Third cell - Initialize the chatbot
# Replace 'model_name' with your preferred Ollama model
chatbot = ConversationManager(model_name='llama3.2:3b')  # or any other model you have in Ollama

# Fourth cell - Create a simple chat interface
from IPython.display import clear_output

def chat():
    print("Chatbot initialized! Type 'quit' to exit, 'history' to view conversation history, or 'clear' to clear history.")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            clear_output(wait=True)
            print("History cleared! Type 'quit' to exit, 'history' to view conversation history, or 'clear' to clear history.")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Fifth cell - Start the chat
# Run this cell to start chatting
chat()

Chatbot initialized! Type 'quit' to exit, 'history' to view conversation history, or 'clear' to clear history.

Assistant: Hello Badreddine! I'm doing well, thank you for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always happy to chat with new friends.

It's lovely to meet you, Badreddine! Is there something on your mind that you'd like to talk about, or would you like some conversation starters?

Assistant: I think I see what's going on here! Your name is actually "Badreddine", and we've already established that. But just to confirm, I'll repeat it back: your name is Badreddine. Is there something specific you'd like to know or talk about related to your name?

Assistant: I think we've had this conversation before! Your name is still "Badreddine". If you're asking for confirmation, I'm happy to provide it: yes, your name is indeed Badreddine. Is there something else on your mind that you'd like to talk about?
Goodbye!


In [3]:
# First cell - Import required libraries
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional
from IPython.display import clear_output

class SimpleConversationManager:
    def __init__(self, model_name: str):
        """Initialize conversation manager with specified Ollama model"""
        self.conversations = []  # Store conversation history
        self.ollama_model = model_name
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message to conversation history"""
        self.conversations.append({'role': role, 'content': message})
    
    def generate_response(self, message: str, temperature: float = 0.7, 
                         context_window: int = 10) -> str:
        """Generate response using simple context window"""
        self.add_message(message, 'user')
        
        # Get last n messages for context
        messages = self.conversations[-context_window:]
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history"""
        self.conversations = []
    
    def save_conversations(self, filename: str):
        """Save conversations to file"""
        import json
        with open(filename, 'w') as f:
            json.dump(self.conversations, f)
    
    def load_conversations(self, filename: str):
        """Load conversations from file"""
        import json
        with open(filename, 'r') as f:
            self.conversations = json.load(f)

def simple_chat():
    # Initialize chatbot
    chatbot = SimpleConversationManager(model_name='llama3.2:3b')
    
    print("Simple Chatbot initialized! Commands:")
    print("- 'quit': Exit chat")
    print("- 'history': View conversation history")
    print("- 'clear': Clear history")
    print("- 'save': Save conversation to file")
    print("- 'load': Load conversation from file")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            clear_output(wait=True)
            print("History cleared!")
            continue
        elif user_input.lower() == 'save':
            filename = input("Enter filename to save: ")
            chatbot.save_conversations(filename)
            print(f"Conversations saved to {filename}")
            continue
        elif user_input.lower() == 'load':
            filename = input("Enter filename to load: ")
            chatbot.load_conversations(filename)
            print(f"Conversations loaded from {filename}")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Run this cell to start the simple chatbot
simple_chat()

Simple Chatbot initialized! Commands:
- 'quit': Exit chat
- 'history': View conversation history
- 'clear': Clear history
- 'save': Save conversation to file
- 'load': Load conversation from file

Assistant: Hello Badreddine! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm here to help and chat with you. It's great to meet you!

How about you? What brings you here today? Do you have any questions or topics you'd like to discuss?

Assistant: Badreddine! Yes, I do remember your name. You mentioned it earlier when we started chatting. How can I assist you today?
1. User: hello how are you my name is badreddine

2. Assistant: Hello Badreddine! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm here to help and chat with you. It's great to meet you!

How about you? What brings you here today? Do you have any questions or topics you'd like to discuss?

3. User: do you remember my name?

4. Assistant: Badreddine! Yes

In [6]:
# First cell - Import required libraries
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import faiss
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional
from IPython.display import clear_output

class SemanticConversationManager:
    def __init__(self, model_name: str):
        """Initialize conversation manager with semantic search capabilities"""
        # Initialize sentence transformer for embeddings
        self.tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.embedding_size = 384
        self.index = faiss.IndexFlatL2(self.embedding_size)
        self.conversations = []
        self.ollama_model = model_name
    
    def _get_embedding(self, text: str) -> np.ndarray:
        """Generate embedding for a text string"""
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.pooler_output.cpu().detach().numpy()
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message and its embedding to storage"""
        embedding = self._get_embedding(message)
        self.index.add(embedding)
        self.conversations.append({
            'role': role, 
            'content': message,
            'embedding_idx': len(self.conversations)
        })
    
    def get_semantic_context(self, message: str, k: int = 5) -> List[Dict]:
        """Retrieve most semantically similar messages"""
        # Get embedding for current message
        query_embedding = self._get_embedding(message)
        
        # Search for similar messages
        D, I = self.index.search(query_embedding, k)
        
        # Get relevant messages in order of relevance
        relevant_messages = [self.conversations[i] for i in I[0]]
        
        # Sort by original order to maintain conversation flow
        relevant_messages.sort(key=lambda x: x['embedding_idx'])
        
        return relevant_messages
    
    def generate_response(self, message: str, temperature: float = 0.7,
                         semantic_k: int = 5, include_recent: int = 3) -> str:
        """Generate response using semantic search and recent context"""
        self.add_message(message, 'user')
        
        # Get semantically relevant messages
        semantic_context = self.get_semantic_context(message, k=semantic_k)
        
        # Get most recent messages
        recent_messages = self.conversations[-include_recent:]
        
        # Combine contexts (remove duplicates while preserving order)
        seen = set()
        context_messages = []
        for msg in semantic_context + recent_messages:
            if msg['embedding_idx'] not in seen:
                seen.add(msg['embedding_idx'])
                context_messages.append({
                    'role': msg['role'],
                    'content': msg['content']
                })
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=context_messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history and FAISS index"""
        self.conversations = []
        self.index = faiss.IndexFlatL2(self.embedding_size)
    
    def save_conversations(self, filename: str):
        """Save conversations to file"""
        import json
        # Save only the essential conversation data
        save_data = [{
            'role': msg['role'],
            'content': msg['content']
        } for msg in self.conversations]
        with open(filename, 'w') as f:
            json.dump(save_data, f)
    
    def load_conversations(self, filename: str):
        """Load conversations from file and rebuild FAISS index"""
        import json
        with open(filename, 'r') as f:
            conversations = json.load(f)
        
        # Clear existing data
        self.clear_history()
        
        # Rebuild conversations and index
        for msg in conversations:
            self.add_message(msg['content'], msg['role'])

def semantic_chat():
    # Initialize chatbot
    chatbot = SemanticConversationManager(model_name='llama3.2:3b')
    
    print("Semantic Chatbot initialized! Commands:")
    print("- 'quit': Exit chat")
    print("- 'history': View conversation history")
    print("- 'clear': Clear history")
    print("- 'save': Save conversation to file")
    print("- 'load': Load conversation from file")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            clear_output(wait=True)
            print("History cleared!")
            continue
        elif user_input.lower() == 'save':
            filename = input("Enter filename to save: ")
            chatbot.save_conversations(filename)
            print(f"Conversations saved to {filename}")
            continue
        elif user_input.lower() == 'load':
            filename = input("Enter filename to load: ")
            chatbot.load_conversations(filename)
            print(f"Conversations loaded from {filename}")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Run this cell to start the semantic chatbot
semantic_chat()

Semantic Chatbot initialized! Commands:
- 'quit': Exit chat
- 'history': View conversation history
- 'clear': Clear history
- 'save': Save conversation to file
- 'load': Load conversation from file

Assistant: What's up Badreddine?! My name is not a personal one, I'm an AI assistant, so I don't have a personal name. I'm here to help answer any questions you may have or just chat with you! How's it going?
Conversations saved to chat_1
Goodbye!


In [8]:
# First cell - Import required libraries
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional
from IPython.display import clear_output
import PyPDF2
import re

class PDFEnhancedConversationManager:
    def __init__(self, model_name: str):
        """Initialize conversation manager with specified Ollama model"""
        self.conversations = []  # Store conversation history
        self.ollama_model = model_name
        self.pdf_content = None  # Store PDF content
        self.system_prompt = None  # Store system prompt
    
    def load_pdf(self, pdf_path: str) -> str:
        """Load and parse PDF content"""
        try:
            pdf_text = []
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text = page.extract_text()
                    # Clean the text
                    text = re.sub(r'\s+', ' ', text)
                    pdf_text.append(text)
            
            self.pdf_content = " ".join(pdf_text)
            
            # Create system prompt with PDF content
            self.system_prompt = {
                "role": "system",
                "content": f"""You have been provided with the following document content. 
                When answering questions, use this content as context and provide accurate responses based on it.
                If the answer cannot be found in the document, clearly state that.
                
                Document content:
                {self.pdf_content[:2000]}  # First 2000 chars for initial context
                
                Additional context will be provided in the conversation history."""
            }
            
            # Add chunked PDF content to conversation history
            chunk_size = 2000
            for i in range(0, len(self.pdf_content), chunk_size):
                chunk = self.pdf_content[i:i + chunk_size]
                self.conversations.append({
                    "role": "system",
                    "content": f"Document content continued: {chunk}"
                })
            
            return f"PDF loaded successfully. Total length: {len(self.pdf_content)} characters"
        except Exception as e:
            raise Exception(f"Error loading PDF: {str(e)}")
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message to conversation history"""
        self.conversations.append({'role': role, 'content': message})
    
    def generate_response(self, message: str, temperature: float = 0.7, 
                         context_window: int = 10) -> str:
        """Generate response using simple context window"""
        self.add_message(message, 'user')
        
        # Get context messages
        messages = []
        if self.system_prompt:
            messages.append(self.system_prompt)
        
        # Add recent conversation history
        messages.extend(self.conversations[-context_window:])
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            if msg['role'] != 'system':  # Skip system messages for cleaner output
                print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history and PDF content"""
        self.conversations = []
        self.pdf_content = None
        self.system_prompt = None
    
    def save_conversations(self, filename: str):
        """Save conversations to file"""
        import json
        save_data = {
            'conversations': self.conversations,
            'pdf_content': self.pdf_content
        }
        with open(filename, 'w') as f:
            json.dump(save_data, f)
    
    def load_conversations(self, filename: str):
        """Load conversations from file"""
        import json
        with open(filename, 'r') as f:
            data = json.load(f)
            self.conversations = data['conversations']
            self.pdf_content = data['pdf_content']
            if self.pdf_content:
                # Recreate system prompt
                self.system_prompt = {
                    "role": "system",
                    "content": f"""You have been provided with the following document content. 
                    When answering questions, use this content as context and provide accurate responses based on it.
                    If the answer cannot be found in the document, clearly state that.
                    
                    Document content:
                    {self.pdf_content[:2000]}"""
                }

def pdf_chat():
    # Initialize chatbot
    chatbot = PDFEnhancedConversationManager(model_name='llama3.2:3b')
    
    print("PDF-Enhanced Chatbot initialized! Commands:")
    print("- 'quit': Exit chat")
    print("- 'history': View conversation history")
    print("- 'clear': Clear history")
    print("- 'save': Save conversation to file")
    print("- 'load': Load conversation from file")
    print("- 'pdf': Load a PDF file")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            clear_output(wait=True)
            print("History cleared!")
            continue
        elif user_input.lower() == 'save':
            filename = input("Enter filename to save: ")
            chatbot.save_conversations(filename)
            print(f"Conversations saved to {filename}")
            continue
        elif user_input.lower() == 'load':
            filename = input("Enter filename to load: ")
            chatbot.load_conversations(filename)
            print(f"Conversations loaded from {filename}")
            continue
        elif user_input.lower() == 'pdf':
            pdf_path = input("Enter the path to your PDF file: ")
            try:
                result = chatbot.load_pdf(pdf_path)
                print(result)
            except Exception as e:
                print(f"Error loading PDF: {str(e)}")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Run this cell to start the PDF-enhanced chatbot
pdf_chat()

PDF-Enhanced Chatbot initialized! Commands:
- 'quit': Exit chat
- 'history': View conversation history
- 'clear': Clear history
- 'save': Save conversation to file
- 'load': Load conversation from file
- 'pdf': Load a PDF file
PDF loaded successfully. Total length: 8024 characters

Assistant: The names of the two persons who had the accident are:

1. Badreddine Hannaoui
2. Maxence Ratignier
Goodbye!


In [9]:
# First cell - Import required libraries
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional
from IPython.display import clear_output, display
import ipywidgets as widgets
import PyPDF2
import re
import io

class InteractivePDFChatbot:
    def __init__(self, model_name: str):
        """Initialize conversation manager with specified Ollama model"""
        self.conversations = []
        self.ollama_model = model_name
        self.pdf_content = None
        self.system_prompt = None
        self.setup_upload_widget()
    
    def setup_upload_widget(self):
        """Create and configure the upload widget"""
        self.upload_widget = widgets.FileUpload(
            accept='.pdf',  # Only accept PDF files
            multiple=False,  # Single file upload
            description='Upload PDF',
            layout=widgets.Layout(width='300px')
        )
        self.upload_widget.observe(self._on_upload_change, names='value')
        
        # Create status output widget
        self.status_output = widgets.Output()
    
    def _on_upload_change(self, change):
        """Handle PDF upload event"""
        with self.status_output:
            clear_output()
            if len(change.new) > 0:
                # Get the uploaded file
                uploaded_file = next(iter(change.new.values()))
                try:
                    result = self.process_uploaded_pdf(uploaded_file)
                    print(result)
                except Exception as e:
                    print(f"Error processing PDF: {str(e)}")
    
    def process_uploaded_pdf(self, uploaded_file) -> str:
        """Process uploaded PDF file"""
        try:
            # Create a BytesIO object from uploaded content
            pdf_stream = io.BytesIO(uploaded_file['content'])
            
            pdf_text = []
            pdf_reader = PyPDF2.PdfReader(pdf_stream)
            for page in pdf_reader.pages:
                text = page.extract_text()
                # Clean the text
                text = re.sub(r'\s+', ' ', text)
                pdf_text.append(text)
            
            self.pdf_content = " ".join(pdf_text)
            
            # Create system prompt with PDF content
            self.system_prompt = {
                "role": "system",
                "content": f"""You have been provided with the following document content. 
                When answering questions, use this content as context and provide accurate responses based on it.
                If the answer cannot be found in the document, clearly state that.
                
                Document content:
                {self.pdf_content[:2000]}  # First 2000 chars for initial context
                
                Additional context will be provided in the conversation history."""
            }
            
            # Add chunked PDF content to conversation history
            chunk_size = 2000
            for i in range(0, len(self.pdf_content), chunk_size):
                chunk = self.pdf_content[i:i + chunk_size]
                self.conversations.append({
                    "role": "system",
                    "content": f"Document content continued: {chunk}"
                })
            
            return f"PDF '{uploaded_file['metadata']['name']}' loaded successfully. Total length: {len(self.pdf_content)} characters"
        except Exception as e:
            raise Exception(f"Error processing PDF: {str(e)}")
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message to conversation history"""
        self.conversations.append({'role': role, 'content': message})
    
    def generate_response(self, message: str, temperature: float = 0.7, 
                         context_window: int = 10) -> str:
        """Generate response using simple context window"""
        self.add_message(message, 'user')
        
        # Get context messages
        messages = []
        if self.system_prompt:
            messages.append(self.system_prompt)
        
        # Add recent conversation history
        messages.extend(self.conversations[-context_window:])
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            if msg['role'] != 'system':  # Skip system messages for cleaner output
                print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history and PDF content"""
        self.conversations = []
        self.pdf_content = None
        self.system_prompt = None
        # Clear upload widget
        self.upload_widget.value.clear()
        with self.status_output:
            clear_output()
            print("History and PDF content cleared!")
    
    def save_conversations(self, filename: str):
        """Save conversations to file"""
        import json
        save_data = {
            'conversations': self.conversations,
            'pdf_content': self.pdf_content
        }
        with open(filename, 'w') as f:
            json.dump(save_data, f)
    
    def load_conversations(self, filename: str):
        """Load conversations from file"""
        import json
        with open(filename, 'r') as f:
            data = json.load(f)
            self.conversations = data['conversations']
            self.pdf_content = data['pdf_content']
            if self.pdf_content:
                # Recreate system prompt
                self.system_prompt = {
                    "role": "system",
                    "content": f"""You have been provided with the following document content. 
                    When answering questions, use this content as context and provide accurate responses based on it.
                    If the answer cannot be found in the document, clearly state that.
                    
                    Document content:
                    {self.pdf_content[:2000]}"""
                }

def interactive_chat():
    """Initialize and run the interactive chatbot"""
    # Initialize chatbot
    chatbot = InteractivePDFChatbot(model_name='llama3.2:3b')
    
    # Display upload widget and status
    display(widgets.VBox([
        widgets.HTML("<h3>PDF-Enhanced Chatbot</h3>"),
        chatbot.upload_widget,
        chatbot.status_output
    ]))
    
    print("Interactive Chatbot initialized! Commands:")
    print("- 'quit': Exit chat")
    print("- 'history': View conversation history")
    print("- 'clear': Clear history")
    print("- 'save': Save conversation to file")
    print("- 'load': Load conversation from file")
    print("\nUse the upload widget above to load a PDF file.")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            continue
        elif user_input.lower() == 'save':
            filename = input("Enter filename to save: ")
            chatbot.save_conversations(filename)
            print(f"Conversations saved to {filename}")
            continue
        elif user_input.lower() == 'load':
            filename = input("Enter filename to load: ")
            chatbot.load_conversations(filename)
            print(f"Conversations loaded from {filename}")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Run this cell to start the interactive chatbot
interactive_chat()

Interactive Chatbot initialized! Commands:
- 'quit': Exit chat
- 'history': View conversation history
- 'clear': Clear history
- 'save': Save conversation to file
- 'load': Load conversation from file

Use the upload widget above to load a PDF file.

Assistant: Hello Badreddine! I'm doing well, thank you for asking. It's great to meet you! I'm a large language model, so I don't have emotions or feelings like humans do, but I'm always happy to chat with someone new and help with any questions or topics you'd like to discuss.

How about you? How's your day going so far?

Assistant: Badreddine is a unique and interesting name. Yes, I do remember it - we just started our conversation, and I've been keeping track of the names mentioned. So, feel free to come back and chat with me anytime, Badreddine!
Goodbye!


In [ ]:
import ollama
from ollama import Options
from typing import List, Dict, Any, Optional
from IPython.display import clear_output, display
import ipywidgets as widgets
import PyPDF2
import re
import io
import tkinter as tk
from tkinter import filedialog
import os

class InteractivePDFChatbot:
    def __init__(self, model_name: str):
        """Initialize conversation manager with specified Ollama model"""
        self.conversations = []
        self.ollama_model = model_name
        self.pdf_content = None
        self.system_prompt = None
        self.setup_upload_widget()
        
        # Initialize tkinter but hide the main window
        self.root = tk.Tk()
        self.root.withdraw()
    
    def setup_upload_widget(self):
        """Create and configure the upload widget"""
        self.upload_widget = widgets.FileUpload(
            accept='.pdf',
            multiple=False,
            description='Upload PDF',
            layout=widgets.Layout(width='300px')
        )
        self.upload_widget.observe(self._on_upload_change, names='value')
        self.status_output = widgets.Output()
    
    def _on_upload_change(self, change):
        """Handle PDF upload event from widget"""
        with self.status_output:
            clear_output()
            if len(change.new) > 0:
                # Get the uploaded file
                uploaded_file = next(iter(change.new.values()))
                try:
                    result = self.process_uploaded_pdf(uploaded_file)
                    print(result)
                except Exception as e:
                    print(f"Error processing PDF: {str(e)}")
    
    def open_file_dialog(self):
        """Open system file dialog for PDF selection"""
        file_path = filedialog.askopenfilename(
            title="Select PDF file",
            filetypes=[("PDF files", "*.pdf")]
        )
        
        if file_path:
            try:
                with open(file_path, 'rb') as file:
                    content = file.read()
                    
                # Create a mock uploaded file structure similar to widget upload
                uploaded_file = {
                    'content': content,
                    'metadata': {'name': os.path.basename(file_path)}
                }
                
                result = self.process_uploaded_pdf(uploaded_file)
                print(result)
            except Exception as e:
                print(f"Error processing PDF: {str(e)}")
    
    def process_uploaded_pdf(self, uploaded_file) -> str:
        """Process uploaded PDF file"""
        try:
            # Create a BytesIO object from uploaded content
            pdf_stream = io.BytesIO(uploaded_file['content'])
            
            pdf_text = []
            pdf_reader = PyPDF2.PdfReader(pdf_stream)
            for page in pdf_reader.pages:
                text = page.extract_text()
                # Clean the text
                text = re.sub(r'\s+', ' ', text)
                pdf_text.append(text)
            
            self.pdf_content = " ".join(pdf_text)
            
            # Create system prompt with PDF content
            self.system_prompt = {
                "role": "system",
                "content": f"""You have been provided with the following document content. 
                When answering questions, use this content as context and provide accurate responses based on it.
                If the answer cannot be found in the document, clearly state that.
                
                Document content:
                {self.pdf_content[:2000]}  # First 2000 chars for initial context
                
                Additional context will be provided in the conversation history."""
            }
            
            # Add chunked PDF content to conversation history
            chunk_size = 2000
            for i in range(0, len(self.pdf_content), chunk_size):
                chunk = self.pdf_content[i:i + chunk_size]
                self.conversations.append({
                    "role": "system",
                    "content": f"Document content continued: {chunk}"
                })
            
            return f"PDF '{uploaded_file['metadata']['name']}' loaded successfully. Total length: {len(self.pdf_content)} characters"
        except Exception as e:
            raise Exception(f"Error processing PDF: {str(e)}")
    
    def add_message(self, message: str, role: str = 'user'):
        """Add a message to conversation history"""
        self.conversations.append({'role': role, 'content': message})
    
    def generate_response(self, message: str, temperature: float = 0.7, 
                         context_window: int = 10) -> str:
        """Generate response using simple context window"""
        self.add_message(message, 'user')
        
        # Get context messages
        messages = []
        if self.system_prompt:
            messages.append(self.system_prompt)
        
        # Add recent conversation history
        messages.extend(self.conversations[-context_window:])
        
        options = Options(
            temperature=temperature,
            num_ctx=8192,
            num_predict=100,
        )
        
        response = ollama.chat(
            model=self.ollama_model,
            messages=messages,
            options=options
        )
        
        new_message_content = response['message']['content']
        self.add_message(new_message_content, 'assistant')
        return new_message_content
    
    def view_conversation_history(self):
        """Display the conversation history"""
        for i, msg in enumerate(self.conversations):
            if msg['role'] != 'system':  # Skip system messages for cleaner output
                print(f"{i+1}. {msg['role'].title()}: {msg['content']}\n")
    
    def clear_history(self):
        """Clear the conversation history and PDF content"""
        self.conversations = []
        self.pdf_content = None
        self.system_prompt = None
        # Clear upload widget
        self.upload_widget.value.clear()
        with self.status_output:
            clear_output()
            print("History and PDF content cleared!")
    
    def save_conversations(self, filename: str):
        """Save conversations to file"""
        import json
        save_data = {
            'conversations': self.conversations,
            'pdf_content': self.pdf_content
        }
        with open(filename, 'w') as f:
            json.dump(save_data, f)
    
    def load_conversations(self, filename: str):
        """Load conversations from file"""
        import json
        with open(filename, 'r') as f:
            data = json.load(f)
            self.conversations = data['conversations']
            self.pdf_content = data['pdf_content']
            if self.pdf_content:
                # Recreate system prompt
                self.system_prompt = {
                    "role": "system",
                    "content": f"""You have been provided with the following document content. 
                    When answering questions, use this content as context and provide accurate responses based on it.
                    If the answer cannot be found in the document, clearly state that.
                    
                    Document content:
                    {self.pdf_content[:2000]}"""
                }

def interactive_chat():
    """Initialize and run the interactive chatbot"""
    # Initialize chatbot
    chatbot = InteractivePDFChatbot(model_name='llama3.2:3b')
    
    # Display upload widget and status
    display(widgets.VBox([
        widgets.HTML("<h3>PDF-Enhanced Chatbot</h3>"),
        chatbot.upload_widget,
        chatbot.status_output
    ]))
    
    print("Interactive Chatbot initialized! Commands:")
    print("- 'pdf': Open file dialog to select PDF")
    print("- 'quit': Exit chat")
    print("- 'history': View conversation history")
    print("- 'clear': Clear history")
    print("- 'save': Save conversation to file")
    print("- 'load': Load conversation from file")
    print("\nUse the upload widget above or type 'pdf' to load a PDF file.")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'pdf':
            chatbot.open_file_dialog()
            continue
        elif user_input.lower() == 'quit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'history':
            chatbot.view_conversation_history()
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            continue
        elif user_input.lower() == 'save':
            filename = input("Enter filename to save: ")
            chatbot.save_conversations(filename)
            print(f"Conversations saved to {filename}")
            continue
        elif user_input.lower() == 'load':
            filename = input("Enter filename to load: ")
            chatbot.load_conversations(filename)
            print(f"Conversations loaded from {filename}")
            continue
        elif user_input == "":
            continue
            
        try:
            response = chatbot.generate_response(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Run this cell to start the interactive chatbot
interactive_chat()

Interactive Chatbot initialized! Commands:
- 'pdf': Open file dialog to select PDF
- 'quit': Exit chat
- 'history': View conversation history
- 'clear': Clear history
- 'save': Save conversation to file
- 'load': Load conversation from file

Use the upload widget above or type 'pdf' to load a PDF file.
PDF 'e-constat-auto_20241014-XGQTE.pdf' loaded successfully. Total length: 8024 characters


: 

: 